In [1]:
import pickle
from encode_utils.rerank_data import rerank_dist, rerank_single
from encode_utils.efficient_rerank import get_effrerank_model, run_comstyle
from encode_utils.sco_funct import weightaddprob, default_scofunct
from encode_utils.mt_scores import get_scores_auto
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
import random
import os
import re
from encode_utils.new_mask_utils import randomsingle, useall
from encode_utils.eval_utils import all_lattice_multi, mean, all_unnoun_multi
from generate_tables import metrics_mapping
from encode_utils.rerank_data import rerank_df, rerank_single, rerank_rand, rerank_weighted
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

2022-12-31 11:36:31.856009: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-31 11:36:31.856032: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# run pipeline for generating lattice preds, using different parameters

In [17]:
# set up information for set
col = {
    "noun_xsum": ["nounsum_reversed/", "nounxsumlargeexplodev2.csv"],
    "noun_fren": ["frtest_reversed/", "nounlargeexplodev1.csv"],
    "mt_fren": ["frtest_reversed/", "frenchlargeexplodev1.csv"],
    "mt_ende": ["detest_reversed/", "germanlargeexplodev1.csv"],
    "mt_ende_b12": ["reversed_mtende_beam12/", "mtendebeam12.csv"],
    "mt_ende_b50": ["reversed_mtende_beam50/", "mtendebeam50.csv"],
    "mt_enru": ["rutest_reversed/", "russianlargeexplodev1.csv"],
    "mt_enru_b12": ["reversed_mtenru_beam12/", "mtenrubeam12.csv"],
    "mt_enru_b50": ["reversed_mtenru_beam50/", "mtenrubeam50.csv"],
    "mt_fren_b12": ['reversed_mtfren_beam12/', 'mtfrenbeam12v2.csv'],
    "mt_fren_b50": ['reversed_mtfren_beam50/', 'mtfrenbeam50v2.csv'],
    "noun_fren_b12": ['reversed_mtfren_beam12/', 'nounfrenbeam12v2.csv'],
    "noun_fren_b50": ['reversed_mtfren_beam50/', 'nounfrenbeam50v2.csv'],
    "noun_xsum_b12": ['reversed_xsum_beam12/', 'nounxsumbeam12v2.csv'],
    "noun_xsum_b50": ['reversed_xsum_beam50/', 'nounxsumbeam50v2.csv'],
}
curcol = "mt_enru_b50"
gsuffix = col[curcol][0]
expl_fname = col[curcol][1]
base = "outputs/graph_pickles/"+gsuffix
if "noun" in curcol:
    goldmetric = "utnoun"
else:
    goldmetric = "dupcqe"
explode_df = pd.read_csv("outputs/score_csvs/"+expl_fname)
#TODO switch back for french-english
if "fren" in curcol:
    SETLEN = len(os.listdir(base))
else:
    SETLEN = int(len(os.listdir(base))/2)

In [18]:
explode_df['weightsco'] = explode_df['dupcqe']*0 - explode_df['posthoc']*1.25

In [20]:
rerank_df(explode_df, rerank_single, ['comet', 'comet'])

0.7023660698857231

In [19]:
len(explode_df['ref'].unique())

453

In [5]:
# use noun model
if "noun" in expl_fname:
    encodemod = get_effrerank_model("noun")
# use mt model (causal)
else:
    encodemod = get_effrerank_model("comstyle")
xlm_tok = AutoTokenizer.from_pretrained("xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_44/checkpoints/epoch=9-step=40000.ckpt.


freeze embeds


In [6]:
SETLEN = len(os.listdir(base))

In [7]:
args = {
    'setlen':int(SETLEN),
    'tok':xlm_tok, 
    'dev':device,
    'model':encodemod,
    'explode_df':explode_df,
    'base':base,
    'goldmetric':goldmetric,
    'device':device, 
    'efficient':False,
    'noregen':False
}

In [ ]:
apreds = rand_robust(-1, 100, "unique_nouns")

In [16]:
mean(apreds)

11.401666666666667

In [24]:
rerank_df(explode_df, rerank_single, ['utnoun', 'unique_nouns'])

8.251655629139073

In [8]:
def get_eval_df(pred_df, tfr, gold):
    tmp = []
    # get dataframe with all appropriate predictions
    for ind, row in pred_df.iterrows():
        hyplist = literal_eval(row['ahyps'])
        scolist = literal_eval(row['ascos'])
        assert len(hyplist)==len(scolist)
        for i in range(len(hyplist)):
            tmp.append({
                'ref':row['ref'],
                'src':row['src'],
                'hyp':hyplist[i],
                tfr:scolist[i]
            })
    evaldf = pd.DataFrame(tmp)
    # get gold score for the new stuff
    metrics_mapping(gold, evaldf)
    return evaldf    

In [ ]:
latpreds = all_lattice_multi(1, default_scofunct, randomsingle, args)

In [10]:
metrics_mapping("unique_nouns", latpreds)

In [11]:
mean(latpreds['unique_nouns'])

8.20309050772627

In [12]:
print(mean(latpreds['goldsco']))
print(mean(latpreds['hypsco']))
print(mean(latpreds['goldsco'])-mean(latpreds['hypsco']))

0.8548662506915612
0.8488538848445905
0.006012365846970735


In [16]:
rerank_df(explode_df, rerank_rand, ['unique_nouns', 'unique_nouns'])

7.008333333333334

In [46]:
latpreds.to_csv("outputs/predcsvs/mtenru_16_valid.csv")

In [33]:
len(explode_df[explode_df['ref']==worseuns[7]])

12

In [42]:
latpreds['gold'][0] # maybe there's a tokenization mismatch? 

'Portsmouth moved into the League Two play-off places with victory at Notts County thanks to two goals from Luke Bennett.'

In [8]:
ind = 6
print(latpreds['gold'][ind], " ", latpreds['goldsco'][ind])
print(latpreds['hyp'][ind], " ", latpreds['hypsco'][ind])

A simplified CAP would promote cleaner, more productive and more efficient farming.   0.8747031688690186
A simplified CAP would promote cleaner, more productive and more efficient farming.   0.8747031688690186


In [ ]:
for l in list(latpreds['hyp']):
    if l not in explode_df['hyp']:
        print("o")
    else:
        printn("i")

In [30]:
worseuns = latpreds[latpreds['hypsco']<latpreds['goldsco']]['ref'].unique()

In [ ]:
graph = pickle.load(open(base+str(0), 'rb'))

In [27]:
graph['root'].nextlist[1].token_str

'Two'

In [ ]:
graph[3]

In [16]:
(len(explode_df)/1000)/(5*32)

4.2108125

In [18]:
tind = 1
print(latpreds.loc[tind])
explode_df[explode_df['hyp']==latpreds['gold'][tind]]

hyp         Negotiations with Iran on its nuclear programm...
hypsco      tensor(0.7139, device='cuda:2', grad_fn=<SumBa...
gold        Negotiations with Iran on its nuclear programm...
goldsco     tensor(0.7139, device='cuda:2', grad_fn=<SumBa...
ascos                                    [0.7138539552688599]
ahyps       [Negotiations with Iran on its nuclear program...
numnodes                                                  174
src         Les négociations avec l’Iran sur son programme...
ref         As a result, the negotiations with Iran over i...
Name: 1, dtype: object


,Unnamed: 0,src,ref,hyp,dcqeold,comet,cqe,posthoc,dupcqe
4946,4946,Les négociations avec l’Iran sur son programme...,"As a result, the negotiations with Iran over i...",Negotiations with Iran on its nuclear programm...,0.787222,0.807614,0.641687,0.347093,0.764838


In [14]:
print(mean(latpreds['goldsco']))
print(mean(latpreds['hypsco']))
print(mean(latpreds['goldsco'])-mean(latpreds['hypsco']))

0.7068876639732199
0.7001804783446899
0.00670718562853001


In [ ]:
goldsco

In [6]:
metrics_mapping("comet", lpredstmp)

wmt20-comet-da is already in cache.
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.


freeze embeds


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
Predicting DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:04<00:00,  3.28it/s]


TOOK TIME  8.24


In [7]:
mean(lpredstmp['comet'])

0.6508571936298679

In [49]:
lpredstmp = pd.read_csv("outputs/predcsvs/noun_comstyle_xsum_randsing_8shot.csv")

In [41]:
luns = latpreds['ref'].unique()

In [42]:
edf = explode_df[explode_df['ref'].isin(luns)]

In [ ]:
olatpreds = pd.read_csv()

In [12]:
rerank_df(explode_df, rerank_single, ['comet', 'comet'])

0.7609684539601108

In [9]:
gs = list(latpreds['hypsco'])
hs = list(latpreds['goldsco'])

In [10]:
mean([max(gs[i], hs[i]) for i in range(len(latpreds))])

tensor(0.7450, device='cuda:2', grad_fn=<DivBackward0>)

In [ ]:
metrics_mapping

In [9]:
outfile = "mt_fren_eelb501shotweighted.csv"
latpreds.to_csv("outputs/predcsvs/"+outfile)

In [10]:
lpredstmp = pd.read_csv("outputs/predcsvs/"+"nounfren_32_valid.csv")

In [22]:
lpredstmp.loc[0]['ascos']

'[1.053318738937378, 1.053318738937378, 1.053318738937378, 1.0222697257995605, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.0222697257995605, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.053318738937378, 1.0846971273422241]'

In [14]:
downstream = get_eval_df(lpredstmp, "utnoun", "unique_nouns")

In [15]:
def robust_rerank_preds(rermet, goldmet, rerdf, samp_n, runs):
    alluns = rerdf['ref'].unique()
    scodist = []
    for a in alluns:
        tmpdf = rerdf[rerdf['ref']==a]
        rerl = list(zip(tmpdf[rermet], tmpdf[goldmet]))
        tot = 0
        for r in range(runs):
            samp = random.choices(rerl, k=samp_n)
            bind = np.argmax([s[0] for s in samp])
            tot+= samp[bind][1]
        scodist.append(tot/runs)
    return scodist

In [17]:
mean(robust_rerank_preds('utnoun', 'unique_nouns', downstream, 8, 1000))

9.085687089715538

In [25]:
downstream

,ref,src,hyp,utnoun,unique_nouns
0,Everton's ambition to build a new Â£300m stadi...,Liverpool Mayor Joe Anderson said it was a gre...,"Liverpool City Council has said it is ""job don...",1.053319,11
1,Everton's ambition to build a new Â£300m stadi...,Liverpool Mayor Joe Anderson said it was a gre...,"Liverpool City Council has said it is ""job don...",1.053319,11
2,Everton's ambition to build a new Â£300m stadi...,Liverpool Mayor Joe Anderson said it was a gre...,"Liverpool City Council has said it is ""job don...",1.053319,11
3,Everton's ambition to build a new Â£300m stadi...,Liverpool Mayor Joe Anderson said it was a gre...,"Liverpool City Council has said it is ""job don...",1.022270,11
4,Everton's ambition to build a new Â£300m stadi...,Liverpool Mayor Joe Anderson said it was a gre...,"Liverpool City Council has said it is ""job don...",1.053319,11
...,...,...,...,...,...
19195,"Over the last week, I've had a taste of our mo...",I've downloaded an episode of David Attenborou...,I've been using a mobile phone to watch TV for...,0.720445,7
19196,"Over the last week, I've had a taste of our mo...",I've downloaded an episode of David Attenborou...,I've been using a mobile phone to watch TV for...,0.720445,7
19197,"Over the last week, I've had a taste of our mo...",I've downloaded an episode of David Attenborou...,I've been able to watch TV shows and movies at...,0.633280,5
19198,"Over the last week, I've had a taste of our mo...",I've downloaded an episode of David Attenborou...,I've been able to watch TV shows and movies at...,0.633280,5


In [12]:
import time
import random
from ast import literal_eval

In [13]:
def rand_robust(samp_n, numruns, metric):
    alluns = explode_df['ref'].unique()
    avg_preds = []
    # repeat for everything in explode_df
    for a in alluns:
        if type(a) is not str:
            continue
        if len(a)>5:
            scolist = explode_df[explode_df['ref']==a]
            totsco = 0
            if samp_n>0:
                for n in range(numruns):
                    samp = scolist.sample(samp_n, replace=True)
                    totsco+=max(list(samp[metric]))
                avg_preds.append(totsco/numruns)
                #print(totsco/numruns)
            else:
                avg_preds.append(max(list(scolist[metric])))
            if len(avg_preds)%30==0:
                print(avg_preds[-1])
                
    return avg_preds
        
def eel_robust(samp_n, numruns, df):
    total = []
    for index, row in df.iterrows():
        l = literal_eval(row['ascos'])
        tot = 0
        for n in range(numruns):
            tot+=max(random.choices(l, k=samp_n))
        total.append(tot/numruns)
    return total
            

In [61]:
lpredstmp = pd.read_csv("outputs/predcsvs/"+"mtenru_16_valid.csv")

In [63]:
mean(eel_robust(1, 1000, lpredstmp))

0.8361491533353252

In [17]:
apreds = rand_robust(1, 100, "dupcqe")

0.8044666165113449
0.4537063679099083
0.6423397800326347
0.3611555516719818
0.7103223258256912
0.69183228880167
0.7604464161396026
0.5041212370991707
0.6260834765434266
0.7240940070152283
0.7586746138334274
0.7200856566429138
0.5755285555124283
0.7995218813419342
0.6039111384749413
0.6667162621021271
0.5659683099389077
0.664055059850216
0.6100616631656886
0.6489359080791474
0.8611379724740982
0.6631572848558426
0.5775548872351647
0.5935747650265694
0.13072008468210697
0.6354189884662628
0.49711788311600685
0.6884044453501701
0.7324190041422844
0.6834666362404823
0.11586612567305565
0.5432203797996045


In [18]:
mean(apreds)

0.5816642331320379

In [9]:
explode_df

,Unnamed: 0,src,ref,hyp,utnounprev,unique_nouns,utnoun
0,0,The gunmen had approached the restaurant from ...,Somali security forces have retaken control of...,Somali troops have killed all the militants wh...,0.512872,6,0.542059
1,1,The gunmen had approached the restaurant from ...,Somali security forces have retaken control of...,Somalia's security forces have killed all the ...,0.682395,7,0.720953
2,2,The gunmen had approached the restaurant from ...,Somali security forces have retaken control of...,Somali security forces have killed all the mil...,0.633267,7,0.672587
3,3,The gunmen had approached the restaurant from ...,Somali security forces have retaken control of...,Somali forces have killed all the militants wh...,0.540708,6,0.573508
4,4,The gunmen had approached the restaurant from ...,Somali security forces have retaken control of...,Somali security forces say they have killed al...,0.642842,7,0.678187
...,...,...,...,...,...,...,...
673725,673725,The Canadian singer told BBC Radio 4's Desert ...,Michael Buble has revealed how he went from ba...,Michael Buble has revealed that a Warner Bros ...,0.574919,6,0.613741
673726,673726,The Canadian singer told BBC Radio 4's Desert ...,Michael Buble has revealed how he went from ba...,Michael Buble has revealed how a Warner Bros e...,0.577802,6,0.618353
673727,673727,The Canadian singer told BBC Radio 4's Desert ...,Michael Buble has revealed how he went from ba...,Michael Buble has told how a Warner Bros execu...,0.586550,6,0.630084
673728,673728,The Canadian singer told BBC Radio 4's Desert ...,Michael Buble has revealed how he went from ba...,Michael Buble has described how a Warner Bros ...,0.575857,6,0.618203


In [ ]:
phoc = all_unnoun_multi(32, "dupcqe", args)
print(mean(phoc))

In [ ]:
s = time.time()
args['noregen'] = True
args['setlen'] = 100
phoc = all_unnoun_multi(1, "utnoun", args)
print(mean(phoc))
e = time.time()

In [11]:
(e-s)/len(phoc)

0.9002537321537099

In [ ]:
args['explode_df']

In [37]:
explode_df['phocrank'] = explode_df['posthoc']*-1
#rand = all_unnoun_multi(32)

In [32]:
mean(rand)

tensor(0.8749, device='cuda:2', grad_fn=<DivBackward0>)

In [49]:
latpreds[latpreds['goldsco']-latpreds['hypsco']>.2]

,hyp,hypsco,gold,goldsco,ascos,ahyps,numnodes
7,Немецкие и французские стороны явно нуждались ...,"tensor(0.3987, device='cuda:2', grad_fn=<SumBa...",Германия и Франция явно нуждались во взаимном ...,"tensor(0.9256, device='cuda:2', grad_fn=<SumBa...",[0.3987096846103668],[Немецкие и французские стороны явно нуждались...,385
13,Небольшая часть населения Индии не имеет ни од...,"tensor(0.6347, device='cuda:2', grad_fn=<SumBa...",Сложно понять низкую позицию Индии.,"tensor(0.9250, device='cuda:2', grad_fn=<SumBa...",[0.6347033977508545],[Небольшая часть населения Индии не имеет ни о...,501
19,Так много о предсказках Кругмана.,"tensor(0.2668, device='cuda:2', grad_fn=<SumBa...","Это все, что предсказывал Кругман.","tensor(0.7456, device='cuda:2', grad_fn=<SumBa...",[0.2667738199234009],[Так много о предсказках Кругмана.],554
31,"Мы узнали, что это была просто воздушная пузыр...","tensor(0.8288, device='cuda:2', grad_fn=<SumBa...","Мы поняли, что это было просто кружево.","tensor(1.0661, device='cuda:2', grad_fn=<SumBa...",[0.8288499116897583],"[Мы узнали, что это была просто воздушная пузы...",558
71,В некоторых странах до 30% более оптимистичных...,"tensor(0.7622, device='cuda:2', grad_fn=<SumBa...",До 30% наиболее оптимистичных потребителей в н...,"tensor(1.0325, device='cuda:2', grad_fn=<SumBa...",[0.7622228264808655],[В некоторых странах до 30% более оптимистичны...,381


In [ ]:
# code for getting comparable sets for lattice generation
#frset = pd.read_csv("outputs/score_csvs/frenchlargeexplodev1.csv")
#xsumset = pd.read_csv("outputs/score_csvs/nounxsumlargeexplodev2.csv")
#frset = frset.drop_duplicates(subset='ref')
#xsumset = xsumset.drop_duplicates(subset='ref')
#xsumset.to_csv("sumexamps.csv")
#frset.to_csv("frset.csv")